In [3]:
import os
from pathlib import Path
import pandas as pd

from config_file import config

In [ ]:
dataset_path = config.DATASET_PATH
folds_dir = dataset_path / "5-folds"
os.makedirs(folds_dir, exist_ok=True)
os.makedirs(folds_dir / "validation", exist_ok=True)
os.makedirs(folds_dir / "train", exist_ok=True)

In [37]:
train = pd.read_csv(dataset_path / "train.csv")
games_info = pd.read_csv(dataset_path / "games_info.csv")

In [36]:
train.head()

,file_name,caption
0,train/Assassin_s Creed II/frame_0001.png,a screenshot from a video game shows a man and...
1,train/Assassin_s Creed II/frame_0002.png,a screenshot from a video game shows a woman w...
2,train/Assassin_s Creed II/frame_0003.png,a screenshot from a video game shows a woman w...
3,train/Assassin_s Creed II/frame_0004.png,a screenshot from a video game shows a man and...
4,train/Assassin_s Creed II/frame_0006.png,a screenshot from a video game shows a man and...


In [38]:
games_info.head()

,title,folder,split,frames_count,genres
0,Assassin's Creed,Assassin_s Creed,train,327,"Role-playing (RPG), Adventure"
1,Assassin's Creed II,Assassin_s Creed II,train,779,"Platform, Adventure"
2,Call of Duty 4: Modern Warfare,Call of Duty 4_ Modern Warfare,train,547,"Shooter, Simulator"
3,Crysis,Crysis,train,417,"Shooter, Adventure"
4,Deus Ex: Invisible War,Deus Ex_ Invisible War,train,298,"Shooter, Role-playing (RPG)"


In [94]:
import random
random.seed(1234)

k_folds = 5
games = games_info[games_info["split"] == "train"]["folder"].to_list()
random.shuffle(games)

fold_size = int(len(games) / k_folds)

def get_fold_games(row, games_list):
    filename = Path(row["file_name"])
    game_folder = filename.parts[1]

    return game_folder in games_list

print(games)

for i in range(0, len(games), fold_size):
    val_games = games[i: i + fold_size]
    train_games = games[:i] + games[i + fold_size:]

    print(val_games, train_games)

    val_fold_frames = train[train.apply(get_fold_games, games_list=val_games, axis=1)]
    train_fold_frames = train[train.apply(get_fold_games, games_list=train_games, axis=1)]

    # val_fold_frames.to_csv(folds_dir / "validation" / f"fold_{i // fold_size + 1}.csv", index=False)
    # train_fold_frames.to_csv(folds_dir / "train" / f"fold_{i // fold_size + 1}.csv", index=False)
    

['Devil May Cry 3_ Dante_s Awakening', 'God of War', 'Devil May Cry 2', 'Quake', 'Deus Ex_ Invisible War', 'Dragon Age_ Origins', 'Half-Life 2', 'Half-Life', 'Lineage II', 'Hitman_ Blood Money', 'Grand Theft Auto_ Vice City', 'Resident Evil 4', 'Prince of Persia_ The Sands of Time', 'Need for Speed_ Carbon', 'Final Fantasy X HD Remaster', 'Halo_ Combat Evolved', 'World of Warcraft', 'Star Wars_ Knights of the Old Republic', 'Metal Gear Solid 4_ Guns of the Patriots', 'Tomb Raider II', 'Tomb Raider_ Anniversary', 'Grand Theft Auto_ San Andreas', 'Crysis', 'Tom Clancy_s Splinter Cell_ Chaos Theory', 'Portal', 'Need for Speed_ Underground', 'Assassin_s Creed II', 'Metal Gear Solid 3_ Snake Eater', 'Quake II', 'The Elder Scrolls III_ Morrowind', 'Call of Duty 4_ Modern Warfare', 'Assassin_s Creed', 'Devil May Cry 4', 'System Shock 2']
['Devil May Cry 3_ Dante_s Awakening', 'God of War', 'Devil May Cry 2', 'Quake', 'Deus Ex_ Invisible War', 'Dragon Age_ Origins'] ['Half-Life 2', 'Half-Life'